<a href="https://colab.research.google.com/github/osapii/quickstart-ios/blob/master/Getting_started_with_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [6]:
%%bigquery --project voicy-aa935 dataframe

with playlist_summary as (select s.*
,t.name
from
(select speaker_id
,count(distinct playlist_id) as playlist_num
,avg(chapter_num) as chapter_num
,avg(playlist_length) as playlist_duration
from 
(select speaker_id
,playlist_id
,count(*) as chapter_num
,round(sum(duration)/1000/60,1) as playlist_length
,max(created) as created
from `voicy_rds.t_voice`
group by speaker_id,playlist_id
having cast(created as string) like "%%2018-10%%"
) as s
group by speaker_id
) as s
left join
voicy_rds.t_speaker as t
on
s.speaker_id = t.id
)

select s.*
,t.uu_rank
,t.rank_range
from
playlist_summary as s
left join
(select speaker_id
,count(distinct uuid) as total_play_uu
,rank() over (order by count(distinct uuid) desc) as uu_rank
,case when rank() over (order by count(distinct uuid) desc) < 31 then "top30" else "over30" end as rank_range 
from `voicy_player_play.play_201810*` 
where platform = "ios"
group by speaker_id) as t
on
s.speaker_id  = t.speaker_id

,speaker_id,playlist_num,chapter_num,playlist_duration,name,uu_rank,rank_range
0,2434,1,3.000000,0.800000,Shingo,NaN,None
1,2208,1,4.000000,10.300000,あゆマド,NaN,None
2,1895,1,1.000000,0.100000,飯塚航平,NaN,None
3,2432,1,1.000000,0.700000,サクラ,NaN,None
4,1868,1,12.000000,4.200000,レタスクラブ,NaN,None
5,2303,1,2.000000,15.500000,takesho,NaN,None
6,1906,1,8.000000,42.100000,りを,NaN,None
7,2456,1,1.000000,0.500000,ぽん,NaN,None
8,2461,1,2.000000,1.300000,たまみん,NaN,None
9,2159,1,1.000000,0.100000,ゆんゆん,NaN,None


# Bigqueryの実行

マジックコマンドを使用する事で、colab上からBigQueryのクエリを実行する事ができる

In [1]:
import altair as alt
from vega_datasets import data

chart0 = alt.Chart(dataframe).mark_point().encode(
    x=alt.X('chapter_num:Q',title='1プレイリストあたりのチャプター数'),
    y=alt.Y('playlist_duration:Q',title='1プレイリストの長さ(分)'),
    color='rank_range:N',
    tooltip=['name:N','chapter_num:N','playlist_duration:N'],
    text = 'name'
).properties(title='チャプター数 x 1プレイリストの長さ (TOP30とそれ以外で比較)')

chart1 = alt.Chart(dataframe).mark_bar().encode(
    x=alt.X('chapter_num_round:Q',title = '1プレイリストあたりのチャプター数'),
    y=alt.Y('count(chapter_num_round):Q',title='パーソナリティ数'),
    tooltip = ['count(chapter_num_round)','chapter_num_round']
).properties(title='平均チャプター数xパーソナリティ数')


chart2 = alt.Chart(dataframe).mark_bar().encode(
    x=alt.X('playlist_duration_round:O',title = '1プレイリストあたりの長さ(分)',bin=alt.Bin(maxbins=10)),
    y=alt.Y('count(playlist_duration_round):Q',title='パーソナリティ数')
).properties(title='平均プレイリストの長さ(分)xパーソナリティ数')

chart1 | chart2 | chart0

NameError: ignored